In [1]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
from datetime import datetime
from datetime import timedelta
import pandas as pd
from shapely.geometry import Point, Polygon
from math import radians, sin, cos, sqrt, atan2
import numpy as np

In [ ]:
stops_df = pd.read_csv("stops.csv")
trips_df = pd.read_csv("e_trips.csv")
stop_times_df = pd.read_csv("stop_times.csv")

In [3]:

'''
all columns from the GTFS dataframes

'trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence',
'pickup_type', 'drop_off_type', 'shape_dist_traveled', 'timepoint',
'route_id', 'service_id', 'trip_headsign', 'direction_id', 'block_id',
'shape_id', 'scheduled_trip_id', 'trip_short_name',
'wheelchair_accessible', 'bikes_allowed', 'at_street',
'corner_placement', 'heading', 'location_type', 'on_street',
'parent_station', 'stop_code', 'stop_desc', 'stop_lat', 'stop_lon',
'stop_name', 'stop_position', 'stop_timezone', 'stop_url',
'wheelchair_boarding', 'zone_id'

final result
- trip_headsign
- bus number
- bus stop order
- bus direction? some way to differentiate direction
- stop lat/lon
- wheelchair boarding
- bikes_allowed
- arrival and departure time
'''

# Merge relevant data
merged_df = pd.merge(stop_times_df, trips_df, on='trip_id')
merged_df = pd.merge(merged_df, stops_df, on='stop_id')

merged_df = merged_df[['trip_id','arrival_time', 'departure_time', 'stop_sequence','trip_headsign', 'direction_id','route_id',
       'wheelchair_accessible', 'bikes_allowed', 'stop_lat', 'stop_lon',
       'stop_name']]
merged_df = merged_df.sort_values(by=['arrival_time', 'departure_time'])

all_unique_stops = merged_df.drop_duplicates(subset=[
    'stop_sequence',
    'trip_headsign',
    'direction_id',
    'route_id',
    'wheelchair_accessible',
    'bikes_allowed',
    'stop_lat',
    'stop_lon',
    'stop_name'
])

# Extract unique bus stops with their coordinates and bus numbers
# unique_stops_routes = merged_df[['route_id', 'stop_name', 'stop_lat', 'stop_lon']].drop_duplicates()
# unique_stops = unique_stops_routes[[ 'stop_name', 'stop_lat', 'stop_lon']].drop_duplicates()
all_unique_stops['is_express'] = all_unique_stops['route_id'] >= 900

all_unique_stops.head()

C:\Users\Martin\AppData\Local\Temp\ipykernel_20980\1047423930.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_unique_stops['is_express'] = all_unique_stops['route_id'] >= 900


,trip_id,arrival_time,departure_time,stop_sequence,trip_headsign,direction_id,route_id,wheelchair_accessible,bikes_allowed,stop_lat,stop_lon,stop_name,is_express
410083,2736572_13819,04:16:00,04:16:00,1,7 William Cannon SB,0,7,1,1,30.338721,-97.719548,7072 Easy Wind/St Johns,False
46760,2726327_0637,04:17:00,04:17:00,1,10 Southpark Meadows SB,0,10,1,1,30.340202,-97.691345,Norwood Transit Center - A,False
410084,2736572_13819,04:17:47,04:17:47,2,7 William Cannon SB,0,7,1,1,30.336202,-97.718247,6904 Airport/Lamar,False
46761,2726327_0637,04:18:28,04:18:28,2,10 Southpark Meadows SB,0,10,1,1,30.340454,-97.690058,1113 Rutherford/Furness,False
410085,2736572_13819,04:18:52,04:18:52,3,7 William Cannon SB,0,7,1,1,30.333155,-97.716673,Airport/Guadalupe,False


In [ ]:
# Step 1: Get the max stop_sequence per bus and bus direction
trip_dir_max_seq =  (
    all_unique_stops
    .groupby(['route_id', 'direction_id', 'trip_id'])['stop_sequence']
    .max()
    .reset_index()
    .rename(columns={'stop_sequence': 'max_stop'})
)

trip_dir_max_seq.head(10)

,route_id,direction_id,trip_id,max_stop
0,1,0,2726088_0110,76
1,1,1,2726131_0001,18
2,1,1,2726197_0039,75
3,2,0,2727334_1598,33
4,2,1,2727408_1461,32
5,3,0,2729077_3326,95
6,3,1,2729133_3220,96
7,4,0,2733639_7963,18
8,4,0,2733650_7961,18
9,4,0,2733654_8026,25


In [5]:
# Step 3: Merge to get trip + route + max stop_sequence
trip_route_seq = pd.merge(trip_dir_max_seq, all_unique_stops, on='trip_id')
# Step 2: Drop the unwanted columns and rename
trip_route_seq = trip_route_seq.drop(columns=['direction_id_y', 'route_id_y']).rename(columns={'direction_id_x': 'direction_id', 'route_id_x': 'route_id'})  # Rename "_x" to original

In [6]:
# Step 4: Find the trip with the highest stop_sequence per route
longest_trips = (
    trip_route_seq
    .sort_values('max_stop', ascending=False)  # Sort to prioritize trips with highest max_stop
    .drop_duplicates(['route_id', 'trip_headsign', 'direction_id'], keep='first')  # Keep first occurrence (longest)
    ['trip_id']  # Optional: Keep only needed columns
)

In [7]:
# Step 5: Filter the original DataFrame to keep only these trips
all_unique_stops = all_unique_stops[all_unique_stops['trip_id'].isin(longest_trips)]

In [10]:
all_unique_stops.dtypes

trip_id                   object
arrival_time              object
departure_time            object
stop_sequence              int64
trip_headsign             object
direction_id               int64
route_id                   int64
wheelchair_accessible      int64
bikes_allowed              int64
stop_lat                 float64
stop_lon                 float64
stop_name                 object
is_express                  bool
dtype: object

In [ ]:
all_unique_stops.to_csv("all_unique_stops.csv")